# Prompt for LLMs

In [ ]:
# plain template class

from langchain import PromptTemplate

prompt = PromptTemplate.from_template("Generate a name for the company that produces {product}")
prompt.format(product = "shoes")

In [ ]:
# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a joke.")
no_input_prompt.format()
# -> "Tell me a joke."

In [ ]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["adjective"], template="Tell me a {adjective} joke.")
one_input_prompt.format(adjective="funny")
# -> "Tell me a funny joke."

In [ ]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["adjective", "content"], 
    template="Tell me a {adjective} joke about {content}."
)
multiple_input_prompt.format(adjective="funny", content="chickens")
# -> "Tell me a funny joke about chickens."

In [ ]:
template = "Tell me a {adjective} joke about {content}."

prompt_template = PromptTemplate.from_template(template)
prompt_template.input_variables
# -> ['adjective', 'content']
prompt_template.format(adjective="funny", content="chickens")
# -> Tell me a funny joke about chickens.

# Prompts for Chat models

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [ ]:
template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
prompt=PromptTemplate(
    template="You are a helpful assistant that translates {input_language} to {output_language}.",
    input_variables=["input_language", "output_language"],
)
system_message_prompt_2 = SystemMessagePromptTemplate(prompt=prompt)

assert system_message_prompt == system_message_prompt_2

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat_prompt.format_prompt(input_language="English", output_language="French", text="I love programming.").to_messages()

# Connecting to Feature Store - Using Feast

**Steps:**
- create a prompt template of template. The idea is that, this will be completed by querying the feature store. The output is a prompt template.
- now the process is same as before. Create a model/llm object, create a chain object with that llm and prompt and then run the chain.
- this process is exactly same for any other feature stores like Teckon ...

In [ ]:
import subprocess
from datetime import datetime

import pandas as pd

from feast import FeatureStore
from feast.data_source import PushMode

In [ ]:
def fetch_historical_features_entity_df(store: FeatureStore, for_batch_scoring: bool):
    # Note: see https://docs.feast.dev/getting-started/concepts/feature-retrieval for more details on how to retrieve
    # for all entities in the offline store instead
    entity_df = pd.DataFrame.from_dict(
        {
            # entity's join key -> entity values
            "driver_id": [1001, 1002, 1003],
            # "event_timestamp" (reserved key) -> timestamps
            "event_timestamp": [
                datetime(2021, 4, 12, 10, 59, 42),
                datetime(2021, 4, 12, 8, 12, 10),
                datetime(2021, 4, 12, 16, 40, 26),
            ],
            # (optional) label name -> label values. Feast does not process these
            "label_driver_reported_satisfaction": [1, 5, 3],
            # values we're using for an on-demand transformation
            "val_to_add": [1, 2, 3],
            "val_to_add_2": [10, 20, 30],
        }
    )
    # For batch scoring, we want the latest timestamps
    if for_batch_scoring:
        entity_df["event_timestamp"] = pd.to_datetime("now", utc=True)

    training_df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:avg_daily_trips",
            "transformed_conv_rate:conv_rate_plus_val1",
            "transformed_conv_rate:conv_rate_plus_val2",
        ],
    ).to_df()
    print(training_df.head())


def fetch_online_features(store, source: str = ""):
    entity_rows = [
        # {join_key: entity_value}
        {
            "driver_id": 1001,
            "val_to_add": 1000,
            "val_to_add_2": 2000,
        },
        {
            "driver_id": 1002,
            "val_to_add": 1001,
            "val_to_add_2": 2002,
        },
    ]
    if source == "feature_service":
        features_to_fetch = store.get_feature_service("driver_activity_v1")
    elif source == "push":
        features_to_fetch = store.get_feature_service("driver_activity_v3")
    else:
        features_to_fetch = [
            "driver_hourly_stats:acc_rate",
            "transformed_conv_rate:conv_rate_plus_val1",
            "transformed_conv_rate:conv_rate_plus_val2",
        ]
    returned_features = store.get_online_features(
        features=features_to_fetch,
        entity_rows=entity_rows,
    ).to_dict()
    for key, value in sorted(returned_features.items()):
        print(key, " : ", value)

In [ ]:
store = FeatureStore(repo_path="./../my_project/feature_repo/")
print("\n--- Run feast apply ---")
subprocess.run(["feast", "apply"])

In [ ]:
from feast import FeatureStore

# You may need to update the path depending on where you stored it
feast_repo_path = "./../my_project/feature_repo/"
store = FeatureStore(repo_path=feast_repo_path)

# import subprocess
# subprocess.run(["feast", "apply"])

In [ ]:
print("\n--- Historical features for training ---")
fetch_historical_features_entity_df(store, for_batch_scoring=False)

print("\n--- Historical features for batch scoring ---")
fetch_historical_features_entity_df(store, for_batch_scoring=True)

print("\n--- Load features into online store ---")
store.materialize_incremental(end_date=datetime.now())

In [ ]:
print("\n--- Online features ---")
fetch_online_features(store)

In [ ]:
print("\n--- Online features retrieved (instead) through a feature service---")
fetch_online_features(store, source="feature_service")

In [ ]:
from langchain.prompts import PromptTemplate, StringPromptTemplate

In [ ]:
template = """Given the driver's up to date stats, write them note relaying those stats to them.
If they have a conversation rate above .5, give them a compliment. Otherwise, make a silly joke about chickens at the end to make them feel better

Here are the drivers stats:
Conversation rate: {conv_rate}
Acceptance rate: {acc_rate}
Average Daily Trips: {avg_daily_trips}

Your response:"""
prompt = PromptTemplate.from_template(template)

In [ ]:
class FeastPromptTemplate(StringPromptTemplate):
    def format(self, **kwargs) -> str:
        driver_id = kwargs.pop("driver_id")
        feature_vector = store.get_online_features(
            features=[
                "driver_hourly_stats:conv_rate",
                "driver_hourly_stats:acc_rate",
                "driver_hourly_stats:avg_daily_trips",
            ],
            entity_rows=[{"driver_id": driver_id}],
        ).to_dict()
        kwargs["conv_rate"] = feature_vector["conv_rate"][0]
        kwargs["acc_rate"] = feature_vector["acc_rate"][0]
        kwargs["avg_daily_trips"] = feature_vector["avg_daily_trips"][0]
        return prompt.format(**kwargs)

In [ ]:
prompt_template = FeastPromptTemplate(input_variables=["driver_id"])

In [ ]:
store.get_online_features(
        features=[
            "driver_hourly_stats:conv_rate",
            "driver_hourly_stats:acc_rate",
            "driver_hourly_stats:avg_daily_trips",
        ],
        entity_rows=[{"driver_id": '1005'}],
    ).to_dict()

In [ ]:
print(prompt_template.format(driver_id=1005))

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import LLMChain

chain = LLMChain(llm=ChatVertexAI(temperature=0.5), prompt=prompt_template)

chain.run(1005)

# Custom Prompt Template

There are two kind of custom prompt templates
- String prompt templates
- chat prompt templates

To create a custom string prompt template, there are two requirements:

- It has an input_variables attribute that exposes what input variables the prompt template expects.
- It exposes a format method that takes in keyword arguments corresponding to the expected input_variables and returns the formatted prompt.

In [ ]:
import inspect
def get_source_code(function_name):
    # Get the source code of the function
    return inspect.getsource(function_name)

In [ ]:
# https://python.langchain.com/docs/modules/model_io/prompts/prompt_templates/custom_prompt_template
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

class FunctionExplainerPromptTemplate(StringPromptTemplate):
    """ a custom prompt template that takes in the function name as input, and formats the prompt template
    to provide the source code of the function"""
    @validator("input_variables")
    def validate_input_variable(cls, v):
        """Validate that the input variables are correct"""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input_variable")
        return v
    def format(self, **kwargs)->str:
        # get the source code of the function
        source_code = get_source_code(kwargs["function_name"])

        # Generate the prompt to be sent to the llm
        prompt = f"""
            Given the function name and the source code, generate an English language explaination of the function.
            Function Name: {kwargs['function_name'].__name__}
            Source Code : 
            {source_code}
            Explaination: 
            """
        return prompt
    
    def _prompt_type(self):
        return "function-explainer"

In [ ]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"
prompt = fn_explainer.format(function_name=get_source_code)
print(prompt)

In [ ]:
print(prompt_template.format(driver_id=1001))

In [ ]:
from langchain.chat_models import ChatVertexAI
from langchain.chains import LLMChain

chain1 = LLMChain(llm=ChatVertexAI(temperature=0.7), prompt=fn_explainer)

chain1.run(function_name=fetch_online_features)

# Few-Shot Prompt Templates

These are prompts with exmples included in the prompt. 

# LLM Integration

List of useful methods:

- LLMs are callable. So you can directly call them like llm("tell me a joke")

- Or you can use the generate() method. generate lets you can call the model with a list of strings, getting back a more complete response than just the text. This complete response can includes things like multiple top responses and other LLM provider-specific information:

In [2]:
from langchain.llms import VertexAI

llm = VertexAI(temperature=0.7)

print(llm("Tell me a joke"))

print(llm.generate(["Tell me a joke"]*2))

Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..
Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/quotas..
Retrying langchain.llms.vertexai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/online_prediction_requests_per_base_model with base model: text-bison. Please submit a quota increase request. https://cloud.google

Why did the scarecrow win an award?
Because he was outstanding in his field!
generations=[[Generation(text='Why did the scarecrow win an award? Because he was outstanding in his field!', generation_info=None)], [Generation(text='Why did the scarecrow win an award?\nBecause he was outstanding in his field!', generation_info=None)]] llm_output=None run=[RunInfo(run_id=UUID('bf6ea96a-4438-44b6-896b-60d613266e9f')), RunInfo(run_id=UUID('f149b5da-1df7-4f2b-b6f1-5d76893d2268'))]
